In [ ]:
!pip install -r requirements.txt
!playwright install

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [3]:
import nest_asyncio
import asyncio
from playwright.async_api import async_playwright

nest_asyncio.apply()

async def scrape_with_playwright(url):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(url)
        content = await page.content()
        await browser.close()
        return content

url = "https://sbjjf.smoothcomp.com/en/federation/16/events/upcoming"

html_content = await scrape_with_playwright(url)

In [4]:
import re

match = re.search(r'var events = (.*)', html_content)
events_data = match.group(1)

In [ ]:
import json

events_list = json.loads(events_data)
vectors = list(map(
    lambda event: f"Event information. name: '{event['title']}', date: {event['eventPeriod']}, place: {event['location_country_human']}, {event['location_city']}",
    events_list))

print(vectors)

In [ ]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma

db = Chroma.from_texts(vectors, OllamaEmbeddings(model = "llama3"))

In [ ]:
from langchain_community.llms import Ollama

llm = Ollama(model = "llama3")

In [8]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
Answer the following question based on the provided context.
<context>
{context}
</context>
Question: {input}""")

In [9]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm, prompt)
retriever = db.as_retriever()

In [11]:
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [ ]:
response = retrieval_chain.invoke({"input": "Give me the name of the event that takes place on December, 01"})
response['answer']